In [1]:
# Exploring the covariance in segregating sites
import numpy as np 
import matplotlib.pyplot as plt
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

# Import the relevant functions / custom classes
import sys 
sys.path.insert(0, '../../src/')
from seg_sites_covar import *
from coal_cov import TwoLocusSimulation, TwoLocusTheoryConstant
from fit_corr_segsites import *
from plot_utils import *

import os
from tqdm import tqdm
from glob import glob

%load_ext autoreload
%autoreload 2

ModuleNotFoundError: No module named 'pyranges'

In [ ]:
plt.rcParams['font.sans-serif'] = "Arial"
plt.rcParams['figure.facecolor'] = "w"
plt.rcParams['figure.autolayout'] = True
from mpl_toolkits.axes_grid1 import make_axes_locatable
    
main_figdir = '../../plots/two_locus_stats/corrSASB_realdata/'
supp_figdir = '../../plots/supp_figs/two_locus_stats/corrSASB_realdata/'
os.makedirs(main_figdir, exist_ok=True)
os.makedirs(supp_figdir, exist_ok=True)

%matplotlib inline

# Monte-Carlo Estimation for Haploid Samples

In [ ]:
# Reading in the data
rec_rate_corr_piA_piB = pd.read_csv('../../results/corr_seg_sites/monte_carlo_est_LBK_UstIshim.csv')
rec_rate_corr_piA_piB.head()

In [ ]:
# Reading in a lot of VCF
pop_1kg_df = pd.read_table('../../data/raw_data/1kg_panel/integrated_call_samples_v3.20130502.ALL.panel')

test_yri_1kg = pop_1kg_df[pop_1kg_df['pop'] == 'YRI']['sample'].values[:3]
test_chb_1kg = pop_1kg_df[pop_1kg_df['pop'] == 'CHB']['sample'].values[:3]
test_ceu_1kg = pop_1kg_df[pop_1kg_df['pop'] == 'CEU']['sample'].values[:3]
test_gih_1kg = pop_1kg_df[pop_1kg_df['pop'] == 'GIH']['sample'].values[:3] 
test_indivs_1kg = np.hstack([test_yri_1kg, test_chb_1kg, test_ceu_1kg, test_gih_1kg])

In [ ]:
mod_1kg = 'NA12383'
anc_ids = ['LBK', 'UstIshim']

In [ ]:
fig, ax = plt.subplots(1,1,figsize=(5,3))
axins = ax.inset_axes([0.5, 0.6, 0.47, 0.4])

# Make theory blue and orange ... 
colors = ['blue', 'orange']
idx = 0
for x in anc_ids:
    cur_df = rec_rate_corr_piA_piB[(rec_rate_corr_piA_piB['ANC_ID'] == x) & (rec_rate_corr_piA_piB['MOD_ID'] == mod_1kg)]
    ax.errorbar(cur_df.rec_rate_mean, cur_df.corr_piA_piB, 
                xerr=2*cur_df.rec_rate_se, yerr=2*cur_df.se_corr_piA_piB, 
                capsize=2, alpha=0.75, color=colors[idx], label=r'%s' % anc_ids[idx]) 
    axins.errorbar(cur_df.rec_rate_mean, cur_df.corr_piA_piB, 
                   xerr=2*cur_df.rec_rate_se, yerr=2*cur_df.se_corr_piA_piB,
                   capsize=2, alpha=0.75, color=colors[idx]) 
    idx += 1
    
    
# Use wattersons estimator on a sample of size 3? 
Ne = 1e4
gen_time = 30.
rec_rates = np.logspace(-5, -3, 1000)
idx = 0
for t in [7e3/gen_time, 45e3/gen_time]:
    corrs_test1 = TwoLocusTheoryConstant._corrSASB(4*Ne*rec_rates, ta=t/Ne/2, theta=4*Ne*1e3*1.2e-8)
    ax.plot(rec_rates, corrs_test1, lw=2, zorder=-1, color=colors[idx], label=r'Theory ($t_a$ = %d gen)' % t)
    axins.plot(rec_rates, corrs_test1, lw=2, zorder=-1, color=colors[idx], label=r'Theory ($t_a$ = %d gen)' % t)
    idx += 1
    
# sub region of the original image
x1, x2, y1, y2 = 5e-5, 3e-4, 0.025, 0.1
axins.set_xlim(x1, x2)
axins.set_ylim(y1, y2)
axins.set_xticklabels('')
axins.set_yticklabels('')
ax.indicate_inset_zoom(axins, label='')  
    
ax.legend(loc=3, fontsize=10, handletextpad=0.1);    
ax.set_xscale('log')
ax.axhline(y=0., linestyle='--', color='black')   
ax.set_xlabel(r'Morgans', fontsize=14)
ax.set_ylabel(r'$Corr(\pi_A,\pi_B)$', fontsize=14)
debox(ax);
plt.savefig(supp_figdir + 'ceu_ustishim_LBK_single_CEU.pdf', dpi=300, bbox_inches='tight')

In [ ]:
# # Computing signed test statistics for this calculation ...  
# _,_, pval_binom = sign_test_corrSASB(corr_haploid_test_gallery[0], corr_haploid_test_gallery[1], test='binom')
# _,_, pval_wilcoxon = sign_test_corrSASB(corr_haploid_test_gallery[0], corr_haploid_test_gallery[1], test='wilcoxon')

# pval_binom, pval_wilcoxon

# Averaging over many modern CEU samples

In [ ]:
fig, ax = plt.subplots(1,1,figsize=(5,3))
axins = ax.inset_axes([0.5, 0.6, 0.47, 0.4])

test_ceu_1kg = pop_1kg_df[pop_1kg_df['pop'] == 'CEU']['sample'].values
anc_ids = ['LBK', 'UstIshim']

# Make theory blue and orange ... 
colors = ['blue', 'orange']
idx = 0
for a in anc_ids:
    # Getting the 
    cur_df_test = rec_rate_corr_piA_piB[(rec_rate_corr_piA_piB.ANC_ID == a) & (np.isin(rec_rate_corr_piA_piB.MOD_ID,test_ceu_1kg))]
    rec_rate_mean = [df_region.rec_rate_mean.values for _, df_region in cur_df_test.groupby('MOD_ID')]
    corr_piA_piB = [df_region.corr_piA_piB.values for _, df_region in cur_df_test.groupby('MOD_ID')]
    
    mean_rec_rates = np.vstack(rec_rate_mean)
    mean_corr_s1_s2 = np.vstack(corr_piA_piB)
    
    ax.errorbar(np.mean(mean_rec_rates, axis=0), np.mean(mean_corr_s1_s2,axis=0),
                xerr=2*np.std(mean_rec_rates, axis=0), yerr=2*np.std(mean_corr_s1_s2, axis=0), 
                capsize=2, alpha=0.75, color=colors[idx], label=r'%s' % a) 

    axins.errorbar(np.mean(mean_rec_rates,axis=0), np.mean(mean_corr_s1_s2,axis=0), 
                xerr=2*np.std(mean_rec_rates,axis=0), yerr=2*np.std(mean_corr_s1_s2, axis=0), 
                capsize=2, alpha=0.75, color=colors[idx], label=r'%s' % a) 
    idx += 1
    

Ne = 1e4
gen_time = 30.
rec_rates = np.logspace(-5, -3, 1000)
idx = 0
for t in [7e3/gen_time, 45e3/gen_time]:
    corrs_test1 = TwoLocusTheoryConstant._corrSASB(4*Ne*rec_rates, ta=t/Ne/2, theta=4*Ne*1e3*1.2e-8)
    ax.plot(rec_rates, corrs_test1, lw=2, zorder=-1, color=colors[idx], label=r'Theory ($t_a$ = %d gen)' % t)
    axins.plot(rec_rates, corrs_test1, lw=2, zorder=-1, color=colors[idx], label=r'Theory ($t_a$ = %d gen)' % t)
    idx += 1
    
# sub region of the original image
x1, x2, y1, y2 = 5e-5, 3e-4, 0.025, 0.1
axins.set_xlim(x1, x2)
axins.set_ylim(y1, y2)
axins.set_xticklabels('')
axins.set_yticklabels('')
ax.indicate_inset_zoom(axins, label='')  
    
ax.legend(loc=3, fontsize=8, handletextpad=0.01);    
ax.set_xscale('log')
# ax.axhline(y=0., linestyle='--', color='black')   
ax.set_xlabel(r'Morgans', fontsize=14)
ax.set_ylabel(r'$Corr(\pi_A,\pi_B)$', fontsize=14); debox(ax);
plt.savefig(main_figdir + 'ceu_ustishim_LBK_trial_all_ceu_samples.pdf', dpi=300, bbox_inches='tight')